In [ ]:
import cv2
import numpy as np
from transformers import AutoFeatureExtractor, AutoModelForObjectDetection
import pandas as pd
import subprocess
from datetime import datetime, timedelta

print("Initializing object detection model...")
# Load the DETR model and feature extractor
extractor = AutoFeatureExtractor.from_pretrained("roupenminassian/detr-resnet-101-finetuned")
model = AutoModelForObjectDetection.from_pretrained("roupenminassian/detr-resnet-101-finetuned")
print("Model initialized!")

# COCO Class labels mapping
COCO_LABELS = {0: 'Car', 1: 'Van', 2: 'Truck', 3: 'Bus', 4: 'Motorbike'}

def map_indices_to_labels(index):
    return COCO_LABELS.get(index, None)

print("Setting up video capture...")
# Load the video for processing
video_path = '/Users/roupenminassian/Downloads/GOPR0069.MP4'
cap = cv2.VideoCapture(video_path)
height, width = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)), int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))

# Define points for perspective transformation
src_points = np.array([
    [274, 720],
    [575, 348],
    [789, 348],
    [1089, 720]
], dtype=np.float32)

dst_width = width
dst_height = int(height * 0.75)  # Adjust this to control the height of the bird's-eye view
dst_points = np.array([
    [0, dst_height],
    [0, 0],
    [dst_width, 0],
    [dst_width, dst_height]
], dtype=np.float32)

M = cv2.getPerspectiveTransform(src_points, dst_points)

# Warp the source points using the transformation matrix
warped_src_points = cv2.perspectiveTransform(src_points.reshape(-1, 1, 2), M).reshape(-1, 2)

# Calculate the pixel distance between the top and bottom points in the transformed frame
warped_distance_pixels = np.linalg.norm(warped_src_points[0] - warped_src_points[3])

# Define lane separation and x-coordinates for lanes
line_separation = dst_width / 5  # Divided by 5 to get 4 lanes
center_x = dst_width / 2 - 25
x_coords = [
    center_x - 1.5 * line_separation,
    center_x - 0.5 * line_separation,
    center_x + 0.5 * line_separation,
    center_x + 1.5 * line_separation
]

def get_lane_from_centroid(cx):
    if cx < x_coords[0]:
        return "Lane 1"
    elif cx < x_coords[1]:
        return "Lane 2"
    elif cx < x_coords[2]:
        return "Lane 3"
    elif cx < x_coords[3]:
        return "Lane 4"
    else:
        return "Lane 5"

def get_video_creation_date(video_path):
    cmd = ['exiftool', '-s', '-s', '-s', '-CreateDate', video_path]
    result = subprocess.run(cmd, capture_output=True, text=True)
    
    if result.returncode == 0:
        return result.stdout.strip()
    else:
        print(f"Error executing exiftool: {result.stderr}")
        return None

print("Fetching video creation date...")
creation_date_str = get_video_creation_date(video_path)
creation_date = datetime.strptime(creation_date_str, '%Y:%m:%d %H:%M:%S')
print(f"Video creation date: {creation_date}")

all_labels = list(COCO_LABELS.values())
all_lanes = ["Lane 1", "Lane 2", "Lane 3", "Lane 4", "Lane 5"]

detections = []
previous_detections = []
conversion_factor = 1
MAX_PERMISSIBLE_DISTANCE = 28
REAL_WORLD_DISTANCE = 28 # in meters
REAL_WORLD_BOX_HEIGHT = 28 # in meters

# Calculate the pixel-to-meter conversion factor
conversion_factor = REAL_WORLD_BOX_HEIGHT / warped_distance_pixels

# Define the number of segments
NUM_SEGMENTS = 10
segment_height = dst_height / NUM_SEGMENTS

# Define the pixel-to-meter ratio at the bottom and top of the image
# You can adjust these based on your knowledge of the scene
ratio_bottom = REAL_WORLD_DISTANCE / dst_height
ratio_top = ratio_bottom * 0.5  # This is an example; adjust as needed

# Calculate the ratio for each segment
segment_ratios = np.linspace(ratio_bottom, ratio_top, NUM_SEGMENTS)

frame_rate = int(cap.get(cv2.CAP_PROP_FPS))
frame_count = 0

print("Starting frame processing...")
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("Video processing completed!")
        break

    timestamp_seconds = frame_count / frame_rate
    current_timestamp = creation_date + timedelta(seconds=timestamp_seconds)
    print(f"\nProcessing frame {frame_count} at timestamp {current_timestamp}...")

    # Transform the frame to bird's-eye view
    warped_frame = cv2.warpPerspective(frame, M, (dst_width, dst_height))

    current_detections = []
    
    frame_rgb = cv2.cvtColor(warped_frame, cv2.COLOR_BGR2RGB)
    inputs = extractor(images=frame_rgb, return_tensors="pt")
    outputs = model(**inputs)

    predicted_logits = outputs.logits.softmax(-1).detach().cpu().numpy()
    predicted_boxes = outputs.pred_boxes[0].detach().cpu().numpy()

    # Create a temporary dictionary to store the counts for this frame
    frame_counts = {(label, lane): 0 for label in all_labels for lane in all_lanes}

    for query in range(predicted_logits.shape[1]):
        label = predicted_logits[0, query].argmax()
        label_name = map_indices_to_labels(label)

        if label_name and predicted_logits[0, query][label] > 0.6 and label_name in COCO_LABELS.values():
            center_x = int(predicted_boxes[query, 0] * dst_width)
            center_y = int(predicted_boxes[query, 1] * dst_height)
            box_width = int(predicted_boxes[query, 2] * dst_width)
            box_height = int(predicted_boxes[query, 3] * dst_height)

            xmin = int(center_x - box_width / 2)
            ymin = int(center_y - box_height / 2)
            xmax = int(center_x + box_width / 2)
            ymax = int(center_y + box_height / 2)

            current_detections.append({
            'label': label_name,
            'centroid': (center_x, center_y),
            'box': (xmin, ymin, xmax, ymax)
            })

            cv2.rectangle(warped_frame, (xmin, ymin), (xmax, ymax), (0, 255, 0), 2)
            cv2.putText(warped_frame, f"{label_name}: {predicted_logits[0, query][label]:.2f}", (xmin, ymin-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

            lane_of_object = get_lane_from_centroid(center_x)
            print(f"Object {label_name} at ({center_x}, {center_y}) is in: {lane_of_object}")

    # Now, calculate speeds by comparing current detections with previous detections
    for curr_det in current_detections:
        closest_prev_det = None
        min_distance = float('inf')

        for prev_det in previous_detections:
            if prev_det['label'] == curr_det['label']:
                distance = np.sqrt((curr_det['centroid'][0] - prev_det['centroid'][0])**2 + 
                                   (curr_det['centroid'][1] - prev_det['centroid'][1])**2)
                if distance < min_distance:
                    min_distance = distance
                    closest_prev_det = prev_det

        if closest_prev_det and min_distance <= MAX_PERMISSIBLE_DISTANCE:
            # Calculate speed in pixels/frame
            speed_pixel_per_frame = min_distance

            # Convert this to real-world speed using frame rate and the conversion factor
            speed_m_per_s = speed_pixel_per_frame * frame_rate * conversion_factor
            speed_km_per_h = speed_m_per_s * 3.6  # Convert m/s to km/h
           
            print(f"Object {curr_det['label']} in {get_lane_from_centroid(curr_det['centroid'][0])} moved at {speed_km_per_h:.2f} km/h")

    # Update previous detections for the next iteration
    previous_detections = current_detections

    # Append the counts (including zeros) to the detections list
    for (label, lane), count in frame_counts.items():
        detections.append({
            'timestamp': current_timestamp,
            'label': label,
            'lane': lane,
            'count': count
        })

    for x in x_coords:
        cv2.line(warped_frame, (int(x), 0), (int(x), dst_height), (255, 0, 0), 2)

    cv2.imshow('Bird\'s-Eye View with Lanes and Detected Objects', warped_frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        print("User interrupted video processing!")
        break

    frame_count += 1

print("Releasing video capture and closing windows...")
cap.release()
cv2.destroyAllWindows()

print("Converting detections to DataFrame...")
# Convert the list of detections to a DataFrame
df = pd.DataFrame(detections)

print("Aggregating data...")
# Aggregate by second, label, and lane to get the average number of vehicles
aggregated_data = df.groupby(['timestamp', 'label', 'lane']).sum().reset_index()

print(aggregated_data)